In [1]:
from clickhouse_driver import Client
import io
import pandas as pd
import numpy as np
from pymongo import MongoClient
from bson.objectid import ObjectId
import datetime
import psycopg2
import requests
import json
import time
from requests_futures.sessions import FuturesSession
from backend_logic import *
session = FuturesSession()

offerid = '5d03c6207e7680141c2a2cf1'
maxRivalsCount = 100000
sqiDiffCoef = 100
maxPos = 10
minCountInSerm = 6

keywords, rivals, startDate, keywordsStr, regionsStr, regions = getRivalsAndData(offerid, maxRivalsCount, sqiDiffCoef, maxPos, minCountInSerm)

In [2]:
import codecs
with codecs.open('rivals.json', 'r', encoding='utf-8') as f:
    file = f.readlines()
    rivals = json.loads(file[0])
len(rivals)

1111

In [3]:
# Получить строку из неконкурентов
notRivals = []
for domain in rivals:
    if domain['isExcluded'] or domain['isOfferDomain']:
        notRivals.append(domain['domain'])

# Получить редко встречающихся конкурентов из базы
query =f"""
    select domain
    from secache3
    where
        keyword in ({keywordsStr})
        and date >= '{startDate}'
        and regionId in ({regionsStr})
        and position <= {maxPos}
        and isMobile = 0
    group by domain
    having count(*) < {minCountInSerm}
"""
client = Client(host = '192.168.10.101', user = 'read_only', password = 'User_ro')
rareRivals = client.execute(query)
client.disconnect()

for rival in rareRivals:
    notRivals.append(rival[0])

notRivalsStr = u"'" + u"', '".join(notRivals) + u"'"

# Получить количество конкурентов по ключам
query =f"""
    select searchEngine, isMobile, keyword, regionId, count(*)
    from secache3
    where
        keyword in ({keywordsStr})
        and date >= '{startDate}'
        and regionId in ({regionsStr})
        and position <= {maxPos}
        and domain not in ({notRivalsStr})
        and isMobile = 0
    group by time, searchEngine, isMobile, keyword, regionId
    having time=max(time)
    order by count(*) desc
"""
client = Client(host = '192.168.10.101', user = 'read_only', password = 'User_ro', settings={'max_query_size':500000})
data = client.execute(query)
client.disconnect()
df = pd.DataFrame(data)
df = df.rename(columns={
    0: "searchEngine",
    1: "isMobile",
    2: "keyword",
    3: "regionId",
    4: "rivalsCount",
})
all_keywords = df.to_dict('records')
for keyword in keywords:
    if len(df[df['keyword'] == keyword['text']].index) == 0:
        all_keywords.append({
            "id": keyword['id'],
            'keyword': keyword['text'],
            "searchEngine": None,
            "isMobile": None,
            "regionId": None,
            "regionName": '—',
            "rivalsCount": 0
        })
for keyword in all_keywords:
    if not 'regionName' in keyword:
        keyword['regionName'] = getRegionName(keyword, regions)


In [6]:
all_keywords, notRivals = getKeywordsRivals(keywords, rivals, maxPos, startDate, keywordsStr, regionsStr, regions, minCountInSerm)
all_keywords

[{'searchEngine': 2,
  'isMobile': 0,
  'keyword': 'кастрюли германия купить в москве',
  'regionId': '1011969;w%2BCAIQICIUTW9zY293LE1vc2NvdyxSdXNzaWE;Moscow',
  'rivalsCount': 10,
  'regionName': 'Москва'},
 {'searchEngine': 2,
  'isMobile': 0,
  'keyword': 'ашан силиконовые формы',
  'regionId': '1011969;w%2BCAIQICIUTW9zY293LE1vc2NvdyxSdXNzaWE;Moscow',
  'rivalsCount': 10,
  'regionName': 'Москва'},
 {'searchEngine': 2,
  'isMobile': 0,
  'keyword': 'форма для выпечки ашан',
  'regionId': '1011969;w%2BCAIQICIUTW9zY293LE1vc2NvdyxSdXNzaWE;Moscow',
  'rivalsCount': 10,
  'regionName': 'Москва'},
 {'searchEngine': 2,
  'isMobile': 0,
  'keyword': 'ашан кастрюли из нержавеющей стали',
  'regionId': '1011969;w%2BCAIQICIUTW9zY293LE1vc2NvdyxSdXNzaWE;Moscow',
  'rivalsCount': 10,
  'regionName': 'Москва'},
 {'searchEngine': 2,
  'isMobile': 0,
  'keyword': 'где купить силиконовые формы для выпечки',
  'regionId': '1011969;w%2BCAIQICIUTW9zY293LE1vc2NvdyxSdXNzaWE;Moscow',
  'rivalsCount': 10,
 

In [7]:
minKeywordRivals = 2
keywordsToDelete = getKeywordsToRemove(all_keywords, minKeywordRivals, regions)
keywordsToDelete

[{'region': 'Москва',
  'keywordsToDelete': ['комплект разделочных досок',
   'герметичные контейнеры для пищевых продуктов',
   'купить сумку-термос в москве',
   'большой ланч бокс',
   'миска пластиковая с крышкой',
   'стеклянный контейнер для еды герметичный',
   'силиконовые формочки для печенья',
   'ваза тиффани купить',
   'купить контейнеры для еды',
   'подставки для ложек и вилок из нержавейки',
   'термос френч пресс',
   'щипцы для спаржи',
   'скатерть с тефлоновым покрытием купить',
   'гейзерная электрическая кофеварка купить',
   'купить термосумку в интернет магазине недорого',
   'контейнер для хранения яиц в холодильнике',
   'механический измельчитель ручной',
   'органайзеры для хранения вещей в шкафу',
   'содасан купить в москве',
   'мантышница цена',
   'где купить термосумку в москве',
   'купить детскую посуду для игры',
   'кастрюли метро',
   'термокружка 1 литр купить',
   'модный термос',
   'термокружка купить недорого в москве',
   'термокружка из нер

In [9]:
filename = 'Rivals ' + offerid + ' ' + 'manual' + '.xlsx'
info = [{
    'Id КП': offerid,
    'Макс. кол-во конкурентов': maxRivalsCount,
    'Макс. отклонение ИКС': sqiDiffCoef,
    'Макс. позиция для поиска конкурентов': maxPos,
    'Мин. кол-во раз, сколько сайт должен встречаться в выдаче, чтобы считаться конкурентом': minCountInSerm,
    'Мин. кол-во конкурентов для запроса': minKeywordRivals
}]
keywordsToDeleteForExcel = []
for item in keywordsToDelete:
    for key in item['keywordsToDelete']:
        keywordsToDeleteForExcel.append({
            'Регион': item['region'],
            'Ключ для удаления': key
        })
writer = pd.ExcelWriter('./downloadable_files/' + filename)
info_df = pd.DataFrame(info)
info_df.to_excel(writer, index=False, sheet_name='Метаданные', freeze_panes = (1,0))
regions_df = pd.DataFrame(regions)
regions_df.to_excel(writer, index=False, sheet_name='Регионы', freeze_panes = (1,0))
rivals_df = pd.DataFrame(rivals)
rivals_df.to_excel(writer, index=False, sheet_name='Конкуренты', freeze_panes = (1,0))
all_keywords_df = pd.DataFrame(keywords)
all_keywords_df.to_excel(writer, index=False, sheet_name='Запросы', freeze_panes = (1,0))
keywordsToDeleteForExcel_df = pd.DataFrame(keywordsToDeleteForExcel)
keywordsToDeleteForExcel_df.to_excel(writer, index=False, sheet_name='Запросы для удаления', freeze_panes = (1,0))
writer.save()
print(filename)

Rivals 5d03c6207e7680141c2a2cf1 manual.xlsx


In [ ]:
import requests
import json
import pandas as pd

url = 'http://192.168.4.158:8901/api/ExportToExcel'

req = requests.post(url, json.dumps({
    'offerUrl':'5cd19cd07e76802420b8ce98',
    'maxRivalsCount':1000,
    'sqiDiffCoef':100,
    'maxPos':10
}, ensure_ascii=False))

rivals_result = req.json()

In [ ]:
import re
import requests
from bs4 import BeautifulSoup
domain = 'vsepodomu.ru'
sitePromise = requests.get('http://' + domain)
html = sitePromise.text
soup = BeautifulSoup(html, 'html.parser')
title = ''
description = ''
icon = 'http://' + domain + '/favicon.ico'
try:
    title = soup.select('title')[0].get_text()
except:
    pass
try:
    description = soup.find("meta", {"name":"description"})['content']
except:
    pass
try:
    icon = soup.find_all(rel = re.compile("Icon|icon"))[0]['href']
    if icon[0] != 'h':
        icon = 'http://' + domain + icon
except:
    pass
res = {
    'title': title,
    'description': description,
    'icon': icon,
    'domain': domain
}
res

In [ ]:
icon = soup.find_all(rel = re.compile("Icon|icon"))
icon

In [5]:
import datetime
mydate = datetime.datetime(2019, 6, 20)
mydate - timedelta(days=1)  

datetime.datetime(2019, 6, 19, 0, 0)